In [1]:
# Basic import
import os
import sys
import json
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import cv2

In [2]:
# Helper function
def writeProgress(msg, count, total):
    sys.stdout.write(msg + "{:.2%}\r".format(count/total))
    sys.stdout.flush()
    
def newPath(path):
    if not os.path.isdir(path):
        os.mkdir(path)

def read_json(src_path):
    with open(src_path, 'r') as json_file:
        data = json.load(json_file)
    return data

def write_json(data,dst_path):
    with open(dst_path, 'w') as outfile:
        json.dump(data, outfile)

## 2019 movies

In [3]:
PATH = './SplitTrailers_crop/'
trailers = os.listdir(PATH)
trailers.sort()
print(trailers[:10])
# trailers = trailers[1:]
print(len(trailers), trailers[:10])

['21bridgesmovie', '47metersdown', 'abeautifuldaymovie', 'abominablemovie', 'adastramovie', 'adogsjourneymovie', 'aftermathmovie', 'aftermovie', 'ahiddenlifefilm', 'alitamovie']
165 ['21bridgesmovie', '47metersdown', 'abeautifuldaymovie', 'abominablemovie', 'adastramovie', 'adogsjourneymovie', 'aftermathmovie', 'aftermovie', 'ahiddenlifefilm', 'alitamovie']


In [4]:
trailers = ["toystory", "doramovie", "meninblack", "angelhasfallen"]

# C_MotionContent
* https://blog.gtwang.org/programming/opencv-motion-detection-and-tracking-tutorial/

In [5]:
SAVE_PATH = './MotionContent_clip/'
newPath(SAVE_PATH)

In [ ]:
C_li = []
for movie in trailers: # [900:]:
    moving_li = []
    print('Running:', movie)
    newPath(SAVE_PATH + movie + '/')
    
    for scene in os.listdir(PATH + movie):
    
        video  = PATH + movie + '/' + scene + '/clip.avi'
        newPath(SAVE_PATH + movie + '/' + scene + '/')
        
        cap = cv2.VideoCapture(video)

        w = int(cap.get(3))
        h = int(cap.get(4))
        fps = cap.get(5)
        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        filename = SAVE_PATH + movie + '/' + scene + '/clip.avi'
        out = cv2.VideoWriter(filename, fourcc, fps, (w, h))

        # 計算畫面面積
        area = w * h

        # 初始化平均影像
        ret, frame = cap.read()

        avg = cv2.blur(frame,(4, 4))
        avg_float = np.float32(avg)
#         plt.imshow(frame)
#         plt.show()

        while(cap.isOpened()):
            # 讀取一幅影格
            ret, frame = cap.read()

            # 若讀取至影片結尾，則跳出
            if ret == False:
                break

            # 模糊處理
            blur = cv2.blur(frame, (4, 4))

            # 計算目前影格與平均影像的差異值
            diff = cv2.absdiff(frame, avg)

            # 將圖片轉為灰階
            gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)

            # 篩選出變動程度大於門檻值的區域
            ret, thresh = cv2.threshold(gray, 25, 255, cv2.THRESH_BINARY)

            # 使用型態轉換函數去除雜訊
            kernel = np.ones((5, 5), np.uint8)
            thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=2)
            thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=2)

            # 產生等高線
            cnts, hierarchy = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            moving = 0
            for c in cnts:
                moving += cv2.contourArea(c)
                
            moving_li.append(moving/area)
            
            cv2.putText(frame, "{:.2f}%".format(moving/area*100), (40, 40), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1.4, (0, 255, 0), 3)

            # 畫出等高線（除錯用）
            cv2.drawContours(frame, cnts, -1, (0, 255, 255), 2)
            out.write(frame)
            img_rgb = frame[:,:,::-1]
            plt.imshow(img_rgb)
            num = str(int(cap.get(1)))
            
            plt.savefig(SAVE_PATH + movie + '/' + scene + '/' + num + '.png', dpi=150)

            # 更新平均影像
            cv2.accumulateWeighted(blur, avg_float, 0.01)
            avg = cv2.convertScaleAbs(avg_float)

        cap.release()
        out.release()
        cv2.destroyAllWindows()

Running: toystory
Running: doramovie
Running: meninblack


KeyboardInterrupt: 